In [1]:
import pickle
import xgboost as xgb
from scipy import stats
from sklearn.model_selection import GridSearchCV
from itertools import chain

In [2]:
with open('data.pkl', 'rb') as file:
    train, test = pickle.load(file)
train.head()

,Reviews,Ratings,Genre,BookCategory,EditionType,EditionMonth,EditionYear,EditionDay,Author1,Author2,...,SynopsisFeatures120,SynopsisFeatures121,SynopsisFeatures122,SynopsisFeatures123,SynopsisFeatures124,SynopsisFeatures125,SynopsisFeatures126,SynopsisFeatures127,SynopsisFeatures128,Price
0,0.409600,0.468746,2,1,6,3.0,2016.0,10.0,0,1,...,233.580963,-218.034454,-69.390533,98.005386,68.660393,-246.383255,-134.644257,-66.794785,100.837883,5.398163
1,0.370151,0.520389,50,3,6,11.0,2012.0,7.0,43,1,...,335.466522,-372.746643,37.508999,12.172302,106.148651,-433.748444,-150.005951,-35.651421,42.431812,5.317777
2,0.849347,0.441125,126,7,6,2.0,1982.0,25.0,0,1,...,419.564941,-519.821655,87.550644,-18.976753,199.813248,-505.484680,-189.340866,-12.567515,44.063568,5.703782
3,0.452122,0.513718,64,6,6,10.0,2017.0,5.0,12,1,...,132.891373,-169.159500,8.444520,12.475374,55.435665,-147.415207,-120.737854,10.605937,29.693066,5.198497
4,1.000000,0.263277,157,2,3,10.0,2006.0,10.0,0,1,...,158.442154,-244.692398,63.849720,-4.507452,94.320618,-210.023666,-227.426117,43.637093,32.485596,6.873805


In [3]:
# drop other cols
droppers = [f'TitleFeatures{i+1}' for i in range(128)] + [f'SynopsisFeatures{i+1}' for i in range(128)]
train.drop(droppers, axis=1, inplace=True)
test.drop(droppers, axis=1, inplace=True)

In [4]:
param_grid = {
    'colsample_bytree': [.5, .7, 0.9, 1.],
    'n_estimators': [50],
    'max_depth': [10, 15, 20, 25],
}

model = xgb.XGBRegressor()
searcher = GridSearchCV(param_grid=param_grid, estimator=model,
            scoring='neg_mean_squared_error', cv=4, verbose=10, n_jobs=-1)


# Fit grid_mse to the data
X, y = train.iloc[:, :-1], train.iloc[:,-1]#.apply(lambda x: np.log(x+ 1))
searcher.fit(X, y)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", searcher.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(searcher.best_score_)))

Fitting 4 folds for each of 16 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   42.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   56.9s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  56 out of  64 | elapsed:  1.6min remaining:   14.0s
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed:  1.9min finished


Best parameters found:  {'colsample_bytree': 0.5, 'max_depth': 10, 'n_estimators': 50}
Lowest RMSE found:  0.6015657761561607


In [5]:
preds = searcher.predict(X)
err = (preds - y)**2
print("RMSE", err.mean() ** .5) #0.2214375717780346

RMSE 0.1158901046975156


In [6]:
if 'Price' in test:
    del(test['Price'])

In [7]:
filename = 'xgboost_gridcv.xlsx'
preds = searcher.predict(test)
preds = np.exp(preds) - 1
submission = pd.DataFrame({'Price': preds})
submission.to_excel(filename, index=False)
print("Saved file to", filename, 'with shape', submission.shape)

Saved file to xgboost_gridcv.xlsx with shape (1560, 1)
